# 데이터 셋 가져오기

# RAP 구조 
"텍스트 문장" + [Triple] (엔티티1, 관계, 엔티티2) 
숨은 의미 찾기 

In [26]:
import pandas as pd 

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

hint = "[Hint] 기계는 감정이 다채롭게, 사람은 감정이 비슷하게 표현현"

train["text"] = train["sentence1"] + " " + hint



## 데이터 분할 

In [27]:
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(
    train["text"], train["label"], test_size=0.2, stratify=train["label"], random_state=42
)


# 전처리 / 모델 불러오기 

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

train_dataset = Dataset.from_pandas(pd.DataFrame({"text": feature_train, "label": target_train}))
test_dataset = Dataset.from_pandas(pd.DataFrame({"text": feature_test, "label": target_test}))

def preprocess(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=5)


Map: 100%|██████████| 10/10 [00:00<00:00, 2667.11 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from transformers import TrainingArguments, Trainer

training_args =TrainingArguments(
                                   output_dir ="./results"
                                 , evaluation_strategy="epoch"
                                 , logging_dir="./logs"
                                 , per_device_eval_batch_size=16
                                 , per_device_train_batch_size=16
                                 , num_train_epochs=3
                                 , learning_rate=5e-5 ,)
trainer = Trainer(
                    model = model
                    , args=training_args
                    , train_dataset=train_dataset
                    , eval_dataset = test_dataset )

trainer.train()

/home/ficl/cuda-env/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 44%|████▍     | 4/9 [00:00<00:00, 12.98it/s]

{'eval_loss': 1.408050537109375, 'eval_runtime': 0.024, 'eval_samples_per_second': 417.489, 'eval_steps_per_second': 41.749, 'epoch': 1.0}


 89%|████████▉ | 8/9 [00:00<00:00, 12.63it/s]

{'eval_loss': 1.3799073696136475, 'eval_runtime': 0.0232, 'eval_samples_per_second': 431.269, 'eval_steps_per_second': 43.127, 'epoch': 2.0}



100%|██████████| 9/9 [00:01<00:00,  4.79it/s]

{'eval_loss': 1.3796817064285278, 'eval_runtime': 0.0191, 'eval_samples_per_second': 523.039, 'eval_steps_per_second': 52.304, 'epoch': 3.0}
{'train_runtime': 1.8801, 'train_samples_per_second': 63.827, 'train_steps_per_second': 4.787, 'train_loss': 1.3950748443603516, 'epoch': 3.0}


TrainOutput(global_step=9, training_loss=1.3950748443603516, metrics={'train_runtime': 1.8801, 'train_samples_per_second': 63.827, 'train_steps_per_second': 4.787, 'total_flos': 15787088547840.0, 'train_loss': 1.3950748443603516, 'epoch': 3.0})

In [30]:
import numpy as np 
from sklearn.metrics import classification_report

# 예측 및 평가
preds = trainer.predict(test_dataset)
y_pred = np.argmax(preds.predictions, axis=1)

print("result")
print(classification_report(target_test, y_pred))

100%|██████████| 1/1 [00:00<00:00, 4275.54it/s]

result
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.40      1.00      0.57         4
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2

    accuracy                           0.40        10
   macro avg       0.10      0.25      0.14        10
weighted avg       0.16      0.40      0.23        10




/home/ficl/cuda-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ficl/cuda-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ficl/cuda-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
